# Stick Game

In [11]:
import numpy as np

## The class to play

In [43]:
class stateSG():
    """
    Class wich represent a state of the game "Stick Game" from the french TV show "Fort Boyard"
    """    
    def __init__(self, nbStick, maxPlayer, nbMaxTook=3):
        """
        Create a state of the game.
        
        Important information for the game is the number of stick (at the beginning), 
        the first player and the max number that a player can take in one turn.
        
        :param nbStick: The number of stick at the beginning of the game
        :param maxPlayer: The player wich begin. True for the "MAX" player and False for "MIN" player
        :param nbMaxTook: The maximum number of sticks that a player can take in one turn
        
        :type nbStick: int
        :type maxPlayer: boolean
        :type maxPlayer: int
        
        :return: The state with the choosen information
        :rtype: stateSG
        """
        self.nbStick = nbStick
        self.maxPlayer = 1 if maxPlayer==True else -1
        self.nbMaxTook = nbMaxTook
        
    def calculateScore(self):
        """
        Calculate the score of the current state if it's a terminal state (then return 0)
        
        This function can be modify to estimate the score for a non terminal state.
        
        :return: The score of the current state
        :rtype: int
        """
        if(self.nbStick <= 1):
            return -self.maxPlayer 
        return 0 #To indicate no one win for now. Can be replace by a function which estimate the score
    
    def getChoices(self):
        """
        Get the different choice for the player for the current state.
        
        :return: Every choices that the player can make. 
        :rtype: list[int]
        """
        nbMaxTurn = self.nbMaxTook if self.nbMaxTook<= self.nbStick else self.nbStick
        return [i for i in range(1, nbMaxTurn+1)]
        
    def doChoice(self, nb, inNewState = False):
        """
        Apply the choice to the current state (inplace or not)
        
        :param nb: The number of stick that the player want to take
        :param inNewState: To choose if the choice is apply inplace (on the current state) or not (on a copy of the current state)
        
        :type nb: int
        :type inNewState: boolean
        
        :return: Nothing if it's inplace then the new state.
        :rtype: stateSG or None
        """
        if(inNewState == False):
            self.nbStick -= nb
            self.maxPlayer *= -1
            return None
        else:
            return stateSG(self.nbStick-nb, -self.maxPlayer)
            
    def undoChoice(self, nb, inNewState = False):
        """
        Undo the given choice for the current state (inplace or not)
        
        :param nb: The number of stick that the player want to take
        :param inNewState: To choose if the choice is apply inplace (on the current state) or not (on a copy of the current state)
        
        :type nb: int
        :type inNewState: boolean
        
        :return: Nothing if it's inplace then the new state.
        :rtype: stateSG or None
        """
        return self.doChoice(-nb, inNewState)
    
    def toKey(self):
        """
        Get the unique ID of the state.
        
        This ID is useful to use memoization in different algorithms
        
        :return: the ID of the current state
        :rtype: string
        """
        return str(self.nbStick)+";"+str(self.maxPlayer)+";"+str(self.nbMaxTook)

## Minmax implementation (without memoization)

In [44]:
def minmaxDecision(state):
    """
    Implementation of MinMax algorithm without memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    
    :type state: StateGame
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        return score, None
    
    choices = state.getChoices()
    
    #MAX player
    if(state.maxPlayer == 1): 
        score = -np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore > score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        return score, bestChoice
    
    #MIN player
    else: 
        score = np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore < score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        return score, bestChoice

In [45]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = minmaxDecision(test) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Negamax implementation (without memoization)

In [46]:
def negamaxDecision(state):
    """
    Implementation of Negamax algorithm without memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    
    :type state: StateGame
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        return score*state.maxPlayer, None
    
    score = -np.inf
    bestChoice = None
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        newScore, newChoice = negamaxDecision(state)
        newScore *= -1
        if(newScore > score):
            score = newScore
            bestChoice = choice
        state.undoChoice(choice)
    return score, bestChoice

In [47]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = negamaxDecision(test)
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Alpha-Beta implementation (without memoization)

In [ ]:
def alphaBetaDecision(state, a, b):
    """
    Implementation of Alphabeta algorithm without memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    :param a: The Alpha parameter of the algorithm
    :param b: The Beta parameter of the algorithm
    
    :type state: StateGame
    :type a: typeof(state.calculScore())
    :type b: typeof(state.calculScore())
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        return score, None
    
    choices = state.getChoices()
    
    #MAX player
    if(state.maxPlayer == 1):
        score = -np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = alphaBetaDecision(state, a, b)
            if(newScore > score):
                score = newScore
                bestChoice = choice
            if(score >= b):
                state.undoChoice(choice)
                return score, bestChoice
            a = max(a, score)
            state.undoChoice(choice)
        return score, bestChoice
    #MIN player
    else:
        score = np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = alphaBetaDecision(state, a, b)
            if(newScore < score):
                score = newScore
                bestChoice = choice
            if(score >= b):
                state.undoChoice(choice)
                return score, bestChoice
            b = min(b, score)
            state.undoChoice(choice)
        return score, bestChoice

In [ ]:
for i in range(1, 11):
    test = stateSG(i, True)
    nb = test.nbStick
    score, choice = alphaBetaDecision(test, -np.inf, np.inf) 
    print("Start with ", nb, " and win with best choice :"+str(choice) if score==1 else " and lose")

## Alpha-Beta Negamax implementation (without memoization)

In [ ]:
def alphaBetaNegamaxDecision(state, ab):
    """
    Implementation of Alphabeta algorithm with "negamax-like" simplification without memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    :param ab: The parameter of the algorithm
    
    :type state: StateGame
    :type ab: typeof(state.calculScore())
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        return score*state.maxPlayer, None
    
    score = -np.inf
    bestChoice = None
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        newScore, newChoice = alphaBetaNegamaxDecision(state, score)
        if(newScore > score):
            score = -newScore
            bestChoice = choice
        if(score <= ab):
            state.undoChoice(choice)
            return score, bestChoice
        state.undoChoice(choice)
    return score, bestChoice

In [ ]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = alphaBetaNegamaxDecision(test, -np.inf) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

## Algorithms with Memoization

In [4]:
states = {}

## Minmax implementation (with memoization)

In [48]:
def minmaxDecision(state):
    """
    Implementation of MinMax algorithm with memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    
    :type state: StateGame
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    global states #global to use memoization
    key = state.toKey()
    
    #if the state already calculated, return values
    if(key in states):
        return states[key]
    
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        states[key] = score, None #Save values
        return states[key]
    
    choices = state.getChoices()
    
    #MAX player
    if(state.maxPlayer == 1):
        score = -np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore > score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        states[key] = score, bestChoice #Save values
        
    #MIN player
    else:
        score = np.inf
        bestChoice = None
        for choice in choices:
            state.doChoice(choice)
            newScore, newChoice = minmaxDecision(state)
            if(newScore < score):
                score = newScore
                bestChoice = choice
            state.undoChoice(choice)
        states[key] = score, bestChoice #Save values
    return states[key]

In [49]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = minmaxDecision(test) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Negamax implementation (with memoization)

In [50]:
def negamaxDecision(state):
    """
    Implementation of Negamax algorithm with memoization
    
    :param state: The state of the game on wich we want to get the best choice and score.
    
    :type state: StateGame
    
    :return: Return a 2-tuple with the score and the best choice to make
    :rtype: 2-tuple(typeof(state.calculScore()), typeof(state.getChoices()[0]))
    """
    global states #global to use memoization
    key = state.toKey()
    
    #if the state already calculated, return values
    if(key in states):
        return states[key]
    
    score = state.calculateScore()
    
    # Terminal Node
    if(score != 0):
        states[key] = score*state.maxPlayer, None #Save values
        return states[key] 
    
    score = -np.inf
    bestChoice = None
    choices = state.getChoices()
    for choice in choices:
        state.doChoice(choice)
        newScore, newChoice = negamaxDecision(state)
        newScore *= -1
        if(newScore > score):
            score = newScore
            bestChoice = choice
        state.undoChoice(choice)
    states[key] = score, bestChoice #Save values
    return states[key]

In [51]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = negamaxDecision(test)
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and win with best choice :2
Start with  8  and win with best choice :3
Start with  9  and lose
Start with  10  and win with best choice :1


## Alpha-Beta implementation (with memoization)

In [39]:
for i in range(1, 11):
    test = stateSG(i, True)
    nb = test.nbStick
    score, choice = alphaBetaDecision(test, -np.inf, np.inf) 
    print("Start with ", nb, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and win with best choice :2
Start with  4  and win with best choice :3
Start with  5  and win with best choice :1
Start with  6  and win with best choice :1
Start with  7  and win with best choice :1
Start with  8  and win with best choice :1
Start with  9  and win with best choice :1
Start with  10  and win with best choice :1


## Alpha-Beta Negamax implementation (with memoization)

In [42]:
for i in range(1, 11):
    test = stateSG(i, True)
    score, choice = alphaBetaNegamaxDecision(test, -np.inf) 
    print("Start with ", test.nbStick, " and win with best choice :"+str(choice) if score==1 else " and lose")

Start with  1  and lose
Start with  2  and win with best choice :1
Start with  3  and lose
Start with  4  and win with best choice :1
Start with  5  and lose
Start with  6  and win with best choice :1
Start with  7  and lose
Start with  8  and win with best choice :1
Start with  9  and lose
Start with  10  and win with best choice :1
